In [10]:
import numpy as np
import numpy.linalg as lg
import math
C=3*10**8
fq=5*10**9
w=2*math.pi*fq
a = 5*10**-2
b = 5*10**-2
c=5*10**-1
numberx=20
numbery=20
numberz=5
m=1
n=0
B0=1
kz=np.sqrt(w*w/C/C-(math.pi*math.pi*(m*m/a/a+n*n/b/b)))
complexcoe=w*w/C/C-kz*kz



def pde_waveguide_TE(f,g,BFx,Domain,Mx,My,Mz,c,tol,MaxIter):
    """
    The PDE solver of u_xx + u_yy + g(x,y)u = f(x,y) over the region
    D = [x0,xf]x[y0,yf] with the boundary conditions:
    u(x0,y) = bx0(y), u(xf,y) = bxf(y)
    u(x,y0) = by0(x), u(x,yf) = byf(x)
    Mx : the number of subintervals along x axis
    My : the number of subintervals along y axis
 
    tol : the tolerance
    MaxIter : the Maximal number of the iteration
     
    The output
    u : u(x_j,y_i)
    x : the uniform grids of x-axis
    y : the uniform grids of y-axis 
    """
    Bx=[]
    By=[]
    Bz=[]
    Ex=[]
    Ey=[]
    Ez=[]
    z=np.linspace(0,c,Mz)
    

    bx0, bxf, by0, byf = BFx[0], BFx[1], BFx[2], BFx[3]
    x0, xf, y0, yf = Domain[0], Domain[1], Domain[2], Domain[3]
     
    hx = float(xf-x0)/Mx
    hy = float(yf-y0)/My

    x = [x0+j*hx for j in range(1,Mx)]
    y = [y0+i*hy for i in range(1,My)]

    Mx, My = Mx+1, My+1
        
    for intz in z:
        
        #set complex number to calculate 
        u = np.zeros([My,Mx])*1j
        F = np.zeros([My,Mx])*1j
        G = np.zeros([My,Mx])*1j
        u0 = np.zeros([My,Mx])*1j
        ux = np.zeros([My,Mx])*1j
        uy = np.zeros([My,Mx])*1j
        ez = np.zeros([My,Mx])*1j
        ex = np.zeros([My,Mx])*1j
        ey = np.zeros([My,Mx])*1j
        

        #set boundary condition
        j = 1
        for xj in x:
            u[0,j], u[My-1,j] = by0(xj), byf(xj)
            j+=1
        i = 1
        for yi in y:
            u[i,0], u[i,Mx-1] = bx0(yi), bxf(yi)
            i+=1
            
        #adding the z direction coefficient 
        u = u*(np.cos((kz*intz))+1j*np.sin(kz*intz))
        
        #initialize as the average of boundary values
        sum_of_bv = sum(u[0,:])+sum(u[My-1,:])+sum(u[1:My-1,0])+sum(u[1:My-1,Mx-1])

        u[1:My-1,1:Mx-1] = float(sum_of_bv)/(2*(Mx+My-2))
        
        
        #set the f(xj,yi) & g(xj,yi)
        for i in range(1,My-1):
            for j in range(1,Mx-1):
                F[i,j], G[i,j] = f(x[j-1],y[i-1]), g(x[j-1],y[i-1])
        
        #define variables
        dx2, dy2 = hx**2, hy**2
        dxy2 = 2*(dx2+dy2)
        rx, ry = dx2/dxy2, dy2/dxy2
        rxy = rx*dy2
        
        
        #solve u,Bz
        for itr in range(MaxIter):
            for i in range(1,My-1):
                for j in range(1,Mx-1):
                    u[i,j] = ry*(u[i,j+1]+u[i,j-1])+rx*(u[i+1,j]+u[i-1,j])+rxy*(G[i,j]*u[i,j]-F[i,j])
             
            Err = abs(u-u0)

            
            if (itr>1) & (Err.max()<tol):
                break
            
            u0=u
    
        
        #calculate Bx,By,Ex,Ey by difference
        A,B=np.gradient(u)
        #print A
        #print B
       
        #solve Bx,By,Ex,Ey
        ux = A*kz*1j/(w*w/C/C-kz*kz)
        uy = B*kz*1j/(w*w/C/C-kz*kz)
        ex = A*w*1j/(w*w/C/C-kz*kz)
        ey = B*(-w)*1j/(w*w/C/C-kz*kz)
        
        #take the real part
        u = np.real(u[1:My-1,1:Mx-1])
        ux = np.real(ux[1:My-1,1:Mx-1])
        uy = np.real(uy[1:My-1,1:Mx-1])
        ex = np.real(ex[1:My-1,1:Mx-1])
        ey = np.real(ey[1:My-1,1:Mx-1])
        ez = np.real(ez[1:My-1,1:Mx-1])
        
        
        Bx.append(ux)
        By.append(uy)
        Bz.append(u)
        Ex.append(ex)
        Ey.append(ey)
        Ez.append(ez)


    return Bx,By,Bz,Ex,Ey,Ez

#--------------------------------------------------------------------------------------------------



BF=[lambda y:B0*np.cos(m*math.pi*0/a)*np.cos(n*math.pi*y/b)
,lambda y:B0*np.cos(m*math.pi*a/a)*np.cos(n*math.pi*y/b)
,lambda x:B0*np.cos(m*math.pi*x/a)*np.cos(n*math.pi*0/b)
, lambda x:B0*np.cos(m*math.pi*x/a)*np.cos(n*math.pi*b/b)]

Domain = [0,a,0,b]

Bx,By,Bz,Ex,Ey,Ez = pde_waveguide_TE(lambda x,y:0.0, lambda x,y:complexcoe, BF,Domain,numberx+1,numbery+1,numberz,c,10**(-8),1000)


#print Bz
#print Bx
#print By

#print Ex
#print Ey
#print Ez



C:\Users\User\Anaconda2\lib\site-packages\ipykernel_launcher.py:88: ComplexWarning: Casting complex values to real discards the imaginary part
